Description:

In 20250425_Biosynthesis_Kinetic_Genes, 20250429_OtherMetabolism_Kinetic_Genes, and 20250501_CellProcesses_Kinetic_Genes, we have attempted to fix DEMs and induce flow to new metabolic reactions added to the model that initially had no flow in 20250307_track_reaction_usage script. Though not all attempts were successful, we can still incorporate the successful updates in a new checkpoint that acts in addition to checkpoint 1. In this checkpoint, we have also attempted triage of ~40 new genes and selected a few (~10) genes that are easy to incorporate, as indicated by green highlight in the Excel file gene_usage_fix_all.xsl. 

The purpose of this script is to incorporate the changes in the whole-cell model, whether it is adding a new transport reaction or novel reactions etc, and see how many more new genes are being used compared to the results in 20250307_track_reaction_usage and checkpoint1. To be more specific, the genes being incorporated into this checkpoint are:

G7408 (yqaB), EG11671 (atoE), EG11724 (adeP), EG10816	(rbsC), EG10818	(rbsK), EG11055	(uidA), EG12393 (ttdT), EG11054 (uhpT), EG10172 (cybB), EG10780 (pspE). 

In [14]:
import numpy as np
import ast
import pandas as pd
import os
import matplotlib.pyplot as plt
import dill
import requests
import xmltodict
import cvxpy as cp
import itertools
import networkx as nx
import plotly.express as px

from scipy.special import logsumexp

%matplotlib inline
# sns.set(style='dar|kgrid', palette='viridis', context='talk')

os.chdir(os.path.expanduser('~/vEcoli'))

from ecoli.processes.metabolism_redux_classic import NetworkFlowModel, FlowResult
from ecoli.library.schema import numpy_schema, bulk_name_to_idx, listener_schema, counts
from ecoli.processes.registries import topology_registry
TOPOLOGY = topology_registry.access("ecoli-metabolism-redux")

In [15]:
# load checkpoint 2 model
time = '400'
date = '2025-07-01'
experiment = 'NEW'
condition = 'basal'
entry = f'{experiment}_{time}_{date}'
folder = f'out/metabolism-comparison/{condition}/{entry}/'

output = np.load(folder + '0_output.npy',allow_pickle='TRUE').item()
# output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
output = output['agents']['0']
fba_cp2 = output['listeners']['fba_results']
bulk_cp2 = pd.DataFrame(output['bulk'])
f = open(folder + 'agent_steps.pkl', 'rb')
agent_cp2 = dill.load(f)
f.close()

In [16]:
# get commonly stored variables
metabolism_cp2 = agent_cp2['ecoli-metabolism-redux-classic']
stoichiometry_cp2 = metabolism_cp2.stoichiometry.copy()
reaction_names_cp2 = metabolism_cp2.reaction_names
fba_new_reaction_ids_cp2 = metabolism_cp2.parameters["fba_new_reaction_ids"]
fba_reaction_ids_to_base_reaction_ids_cp2 = metabolism_cp2.parameters['fba_reaction_ids_to_base_reaction_ids']
metabolites_cp2 = metabolism_cp2.metabolite_names.copy()
binary_kinetic_idx_cp2 = metabolism_cp2.binary_kinetic_idx
exchange_molecules_cp2 = metabolism_cp2.exchange_molecules

S_cp2 = stoichiometry_cp2.copy()
S_cp2 = pd.DataFrame(S_cp2, index=metabolites_cp2, columns=reaction_names_cp2)
homeostatic_count_cp2 = pd.DataFrame(fba_cp2["homeostatic_metabolite_counts"], columns=metabolism_cp2.homeostatic_metabolites).loc[24, :]
homeostatic_cp2 = pd.DataFrame(fba_cp2["target_homeostatic_dmdt"], columns=metabolism_cp2.homeostatic_metabolites).loc[24, :]
maintenance_cp2 = pd.DataFrame(fba_cp2["maintenance_target"][1:], columns=['maintenance_reaction']).iat[24, 0]
kinetic_cp2 = pd.DataFrame(fba_cp2["target_kinetic_fluxes"], columns=metabolism_cp2.kinetic_constraint_reactions).loc[24, :].copy()

In [17]:
# parameters that are the same across the two simulation
kinetic_reaction_ids = metabolism_cp2.kinetic_constraint_reactions
allowed_exchange_uptake = metabolism_cp2.allowed_exchange_uptake
FREE_RXNS = ["TRANS-RXN-145", "TRANS-RXN0-545", "TRANS-RXN0-474"]
ADDED_RXNS = ['HS-TRANSPORT-RXN-CPD0-1202', 'HS-TRANSPORT-RXN-CPD0-1202 (reverse)',
                   'HS-TRANSPORT-RXN[CCO-OUTER-MEM]-OXAMATE', 'HS-TRANSPORT-RXN[CCO-OUTER-MEM]-OXAMATE (reverse)',
                   'HS-TRANSPORT-RXN[CCO-PM-BAC-NEG]-OXAMATE', 'HS-TRANSPORT-RXN[CCO-PM-BAC-NEG]-OXAMATE (reverse)',
                   'HS-BETA-GLUCURONID-RXN_CPD-3611//METOH', 'HS-SPONTANEOUS-TRANSPORT[CCO-OUTER-MEM]-HCN', 'HS-SPONTANEOUS-TRANSPORT[CCO-OUTER-MEM]-HCN (reverse)',
                   'HS-SPONTANEOUS-TRANSPORT[CCO-PM-BAC-NEG]-HCN','HS-SPONTANEOUS-TRANSPORT[CCO-PM-BAC-NEG]-HCN (reverse)']

# Define functions for testing

In [18]:
def get_subset_S(S, met_of_interest):
    S_met = S.loc[met_of_interest, :]
    S_met = S_met.loc[:,~np.all(S_met == 0, axis=0)]
    return S_met, S_met.columns

def test_NetworkFlowModel(objective_weights,
                          uptake_addition = set([]), uptake_removal = set([]), new_exchange_molecules = set([]),
                          add_metabolite = None, add_reaction = None, add_kinetic = None, remove_reaction = None,
                          add_homeostatic_demand = None, force_reaction = None):
    # update exchanges
    uptake = metabolism_cp2.allowed_exchange_uptake.copy()
    uptake = set(uptake)
    uptake = uptake | uptake_addition
    uptake = uptake - uptake_removal

    exchange_molecules = metabolism_cp2.exchange_molecules.copy()
    exchange_molecules = exchange_molecules | new_exchange_molecules

    # update stoichiometry
    reaction_names = metabolism_cp2.reaction_names.copy()
    kinetic_reaction_ids = metabolism_cp2.kinetic_constraint_reactions.copy()
    kinetic = pd.DataFrame(fba_cp2["target_kinetic_fluxes"], columns=metabolism_cp2.kinetic_constraint_reactions).loc[24, :].copy()
    metabolites = metabolism_cp2.metabolite_names.copy()
    homeostatic = pd.DataFrame(fba_cp2["target_homeostatic_dmdt"], columns=metabolism_cp2.homeostatic_metabolites).loc[24, :].copy()
    homeostatic_counts = homeostatic_count_cp2.copy() * metabolism_cp2.counts_to_molar.asNumber()

    S_new = stoichiometry_cp2.copy()

    if add_metabolite is not None: # add to metabolites list because they are currently not included in the model
        for m in add_metabolite:
            if m not in metabolites:
                metabolites.append(m)
        # append rows of zeros to S_new of length add_metabolite
        S_new = np.concatenate((S_new, np.zeros((len(add_metabolite), S_new.shape[1]))), axis=0)
        print(S_new.shape)

    if add_reaction is not None:
        # assert add_reaction is a dictionary
        assert isinstance(add_reaction, dict)

        for r,s in add_reaction.items():
            if r not in reaction_names:
                reaction_names.append(r)
            # append columns of reaction stoich to S_new of length add_reaction
            new_reaction = np.zeros((S_new.shape[0], 1))
            for m, v in s.items():
                new_reaction[metabolites.index(m), 0] = v
            S_new = np.concatenate((S_new, new_reaction), axis=1)
        print(S_new.shape)

    if add_kinetic is not None:
        # assert add_kinetic is a dictionary
        assert isinstance(add_kinetic, dict)

        for r, v in add_kinetic.items():
            if r not in kinetic_reaction_ids:
                kinetic_reaction_ids.append(r)
                kinetic[r] = v

    if remove_reaction is not None:
        for r in remove_reaction:
            r_idx = reaction_names.index(r)
            S_new = np.delete(S_new, r_idx, axis=1)
            reaction_names.remove(r)
            if r in kinetic_reaction_ids:
                kinetic_reaction_ids.remove(r)
                del kinetic[r]

    if force_reaction is not None:
        force_reaction_idx = np.array([reaction_names.index(r) for r in force_reaction])
    else:
        force_reaction_idx = force_reaction

    if add_homeostatic_demand is not None:
        # assert add_homeostatic_demand is a set
        assert isinstance(add_homeostatic_demand, list)

        for met in add_homeostatic_demand:
            homeostatic[met] = 100
            homeostatic_counts[met] = 1

    print(f'Force Reaction ID is:',force_reaction_idx)
    # Solve NetworkFlowModel
    model = NetworkFlowModel(
            stoich_arr=S_new,
            metabolites=metabolites,
            reactions=reaction_names,
            homeostatic_metabolites=list(dict(homeostatic).keys()),
            kinetic_reactions=kinetic_reaction_ids,
            free_reactions=FREE_RXNS)
    model.set_up_exchanges(exchanges=exchange_molecules, uptakes=uptake)
    solution: FlowResult = model.solve(
            homeostatic_concs=homeostatic_counts, # in conc
            homeostatic_dm_targets=np.array(list(dict(homeostatic).values())), # *10^7
            maintenance_target=maintenance_cp2, # *10^6 ish
            kinetic_targets=np.array(list(dict(kinetic).values())), # *10^6 ish
            # binary_kinetic_idx=binary_kinetic_idx, #7646
            binary_kinetic_idx=None,
            force_flow_idx=force_reaction_idx,
            objective_weights=objective_weights, #same
            upper_flux_bound= 1000000000, # increase to 10^9 because notebook runs FlowResult using Counts, WC runs using conc.
            solver=cp.GLOP) #SCS. ECOS
    print(f'Optimal objective function reached is: {solution.objective}')
    return solution.velocities, reaction_names, S_new, metabolites, kinetic

# Create Dataframes to Store Results


In [19]:
# all reactions
sim_cp2 = pd.DataFrame(fba_cp2["estimated_fluxes"], columns= reaction_names_cp2).loc[24, :].copy()

# kinetic reactions
kc_target_cp2 = pd.DataFrame(fba_cp2["target_kinetic_fluxes"], columns= kinetic_reaction_ids).loc[24, :].copy()

In [20]:
df_all = pd.concat([sim_cp2], axis=1).fillna(0)
df_all.columns = ['sim_cp2_basal']

In [21]:
df_all['is_new'] = 'Old Reactions'
df_all.loc[fba_new_reaction_ids_cp2, 'is_new'] = 'New Reactions'
df_all.loc[ADDED_RXNS, 'is_new'] = 'Heena\'s Reactions'


df_all['kinetic'] = [kinetic_cp2[r] if r in kinetic_reaction_ids else False for r in df_all.index]

In [22]:
df_all.loc[['RXN0-5186__G7408-MONOMER','TRANS-RXN0-281', 'TRANS-RXN0-447', 'ABC-28-RXN-ATP/CPD-15829/WATER//ADP/CPD-15829/Pi/PROTON.45.',
            'ABC-28-RXN-ATP/CPD-15829/WATER//ADP/CPD0-1110/Pi/PROTON.45.', 'ABC-28-RXN-ATP/CPD-15829/WATER//ADP/D-Ribopyranose/Pi/PROTON.50.',
            'ABC-28-RXN-ATP/CPD0-1110/WATER//ADP/CPD-15829/Pi/PROTON.45.', 'ABC-28-RXN-ATP/CPD0-1110/WATER//ADP/CPD0-1110/Pi/PROTON.45.',
            'ABC-28-RXN-ATP/CPD0-1110/WATER//ADP/D-Ribopyranose/Pi/PROTON.50.', 'HS-BETA-GLUCURONID-RXN_CPD-3611//METOH',
            'THIOSULFATE-SULFURTRANSFERASE-RXN[CCO-PERI-BAC]-S2O3/HCN//HSCN/SO3/PROTON.40.','HS-SPONTANEOUS-TRANSPORT[CCO-PM-BAC-NEG]-HCN','HS-SPONTANEOUS-TRANSPORT[CCO-PM-BAC-NEG]-HCN (reverse)',
            'THIOSULFATE--THIOL-SULFURTRANSFERASE-RXN', 'TRANS-RXN-127', 'TRANS-RXN-127 (reverse)','TRANS-RXN-33', 'TRANS-RXN0-502', 'TRANS-RXN0-501', 'TRANS-RXN0-534',
            'RXN-20148-CPD-9956/OXYGEN-MOLECULE//SUPER-OXIDE/UBIQUINONE-8/PROTON.58. (reverse)']]

,sim_cp2_basal,is_new,kinetic
RXN0-5186__G7408-MONOMER,0.0,New Reactions,2621.0
TRANS-RXN0-281,0.0,New Reactions,False
TRANS-RXN0-447,0.0,New Reactions,False
ABC-28-RXN-ATP/CPD-15829/WATER//ADP/CPD-15829/Pi/PROTON.45.,0.0,New Reactions,False
ABC-28-RXN-ATP/CPD-15829/WATER//ADP/CPD0-1110/Pi/PROTON.45.,0.0,New Reactions,False
ABC-28-RXN-ATP/CPD-15829/WATER//ADP/D-Ribopyranose/Pi/PROTON.50.,0.0,New Reactions,False
ABC-28-RXN-ATP/CPD0-1110/WATER//ADP/CPD-15829/Pi/PROTON.45.,0.0,New Reactions,False
ABC-28-RXN-ATP/CPD0-1110/WATER//ADP/CPD0-1110/Pi/PROTON.45.,0.0,New Reactions,False
ABC-28-RXN-ATP/CPD0-1110/WATER//ADP/D-Ribopyranose/Pi/PROTON.50.,0.0,New Reactions,False
HS-BETA-GLUCURONID-RXN_CPD-3611//METOH,0.0,Heena's Reactions,False


## Testing

In [106]:
# uptake_addition = set(['ADENOSYLCOBALAMIN[e]', 'COB-I-ALAMIN[e]', 'FERRIC-CITRATE-COMPLEX[e]'])
# # remove_reaction = ['RXN-19341','RXN-19340','RXN-19334']
# uptake_removal = set(['FE+3[e]'])
# add_reaction={
#     'TEMP-TRANS-1565[e] -> [c]': {
#         'COB-I-ALAMIN[c]' : -1,
#         'PROTON[c]': -1,
#         'P3I[c]': -1,
#         'ADENOSYLCOBALAMIN[p]' : 1,
#     },
#     'TEMP-TRANS-1565-2[e] -> [c] (both)': {
#         'FERRIC-CITRATE-COMPLEX[c]' : 1,
#         'FE+3[c]' : -1,
#         'CIT[c]': -1
#     },
#       'TEMP-TRANS-1565-2[e] -> [c] (both)': {
#         'FERRIC-CITRATE-COMPLEX[c]' : -1,
#         'FE+3[c]' : 1,
#         'CIT[c]': 1
#     },
#     # 'TEMP-TRANS-1565-2[e] -> [c] (reverse)': {
#     #     'ADENOSYLCOBALAMIN[p]' : 1,
#     #     'ADENOSYLCOBALAMIN[c]' : -1,
#     # },
# }


uptake_addition = set(['CPD0-1068[e]'])
uptake_removal = set(['Pi[p]'])
objective_weights = {'secretion': 0.01, 'efficiency': 0.000001, 'kinetics': 0.00001}

In [113]:

solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic = test_NetworkFlowModel(
                                            objective_weights,
                                            uptake_addition=uptake_addition,
                                             add_homeostatic_demand=['CPD0-2479[c]'],
                                             uptake_removal=uptake_removal,
                                            # # new_exchange_molecules=new_exchange_molecules,
                                            #  force_reaction=force_reaction,
                                            # add_reaction=add_reaction,
                                            # add_metabolite=add_metabolite,
                                            # remove_reaction=remove_reaction,
)

Force Reaction ID is: None
Optimal objective function reached is: 21378.579154213316


In [114]:
sim_flux = pd.DataFrame({
    'flux': solution_flux,
    'is_new': [
        'New Reactions' if id in fba_new_reaction_ids_cp2
        else 'TEMP' if id in add_reaction.keys()
        else 'Old Reactions'
            for id in test_reaction_names
    ]
}, index=test_reaction_names)

In [116]:
met_of_interest = ['CPD0-2479[c]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
rxns = ['RXN0-6732', 'RXN-17954']
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

,flux,is_new,kinetic
RXN0-6732,68.0,New Reactions,False
RXN-17954,-0.0,New Reactions,False


In [119]:
uptake_addition = set(['TAURINE[e]'])
uptake_removal = set(['GLC[p]'])
force_reaction =['ABC-64-RXN']
# uptake_addition = set(['SORBITOL[e]','MANNITOL[e]'])

#force_reaction = ['RXN-24169', 'RXN-24170', 'RXN-14722']
# uptake_addition = set(['COB-I-ALAMIN[e]', 'CPD-20903[e]', 'Oxidized-Flavoproteins[e]'])

#uptake_addition = set(['CPD-20903[e]','COB-I-ALAMIN[e]'])
# uptake_removal = set(['Pi[p]'])
# force_reaction= ['RXN0-1565']
# force_reaction = ['TRANS-RXN0-283', 'RXN0-1682','RXN0-1721', 'TRANS-RXN-297', 'TRANS-RXN-298']
    # ['TRANS-RXN0-283','RXN0-1682']
# 'BUTYL-HYDROPEROXIDE[p]' 'CPD0-2482[e]', 'CPD0-621[e]', 'CPD0-2241[e]
# add_metabolite = {'CPD-334[e]'}'TRANS-RXN-162'
# remove_reaction = ['RXN0-1565']

# ['RXN0-1565', 'TRANS-RXN0-283', 'RXN0-1682', 'RXN0-1701', 'RXN0-1684', 'RXN0-1702', 'RXN0-1721', 'TRANS-RXN0-629']
# add_reaction = {
#     'TEMP-DTDP-RHAMNOSEs-in[e]->[p]': {
#         'DTDP-RHAMNOSE[e]': -1,
#         'DTDP-RHAMNOSE[p]': 1,
#     },
#     'TEMP-HDTDP-RHAMNOSEs-in[p]->[c]': {
#         'DTDP-RHAMNOSE[p]': -1,
#         'DTDP-RHAMNOSE[c]': 1,
#     }
#
# }
# ['RXN0-262', 'RXN-21601', 'RXN-16457', 'RXN-16456']
# new_exchange_molecules = set(['FE+3[p]', 'N-23-DIHYDROXYBENZOYL-L-SERINE[e]'])
# add_reaction={
#     'TEMP-TRANS-1592[e] -> [c]': {
#         'ADENOSYLCOBALAMIN[e]' : -1,
#         'ADENOSYLCOBALAMIN[p]' : 1,
#     },
#     'TEMP-TRANS-1592-2[e] -> [c] (reverse)': {
#         'ADENOSYLCOBALAMIN[p]' : 1,
#         'ADENOSYLCOBALAMIN[c]' : -1,
#     },
# }
# add_metabolite = {'ADENOSYLCOBALAMIN[e]'}
objective_weights = {'secretion': 0.01, 'efficiency': 0.000001, 'kinetics': 0.00001}

In [120]:

solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic = test_NetworkFlowModel(
                                            objective_weights,
                                            uptake_addition=uptake_addition,
                                            #  add_homeostatic_demand=['ADENOSYLCOBALAMIN[c]'],
                                             uptake_removal=uptake_removal,
                                            # # new_exchange_molecules=new_exchange_molecules,
                                             force_reaction=force_reaction,
                                            # add_reaction=add_reaction,
                                            # add_metabolite=add_metabolite,
                                            # remove_reaction=remove_reaction,
)

Force Reaction ID is: [493]
Optimal objective function reached is: 43373.99632072635


In [121]:
sim_flux = pd.DataFrame({
    'flux': solution_flux,
    'is_new': [
        'New Reactions' if id in fba_new_reaction_ids_cp2
        else 'TEMP' if id in add_reaction.keys()
        else 'Old Reactions'
            for id in test_reaction_names
    ]
}, index=test_reaction_names)

In [122]:
# met_of_interest = ["CPD-20903[p]", 'FERRIC-ENTEROBACTIN-COMPLEX[p]', 'CPD0-2482[p]', 'CPD-25891[c]', 'CPD0-2114[c]','CPD0-2241[c]']
# met_of_interest = ['COB-I-ALAMIN[p]','COB-I-ALAMIN[c]','CPD-1829[c]','CPD-20903[p]', 'Oxidized-Flavoproteins[c]']
# met_of_interest = ['FERRIC-ENTEROBACTIN-COMPLEX[p]', 'CPD-17082[c]','CPD-25891[c]', 'CPD0-2241[c]','CPD0-2241[p]','CPD0-2114[c]','CPD0-621[c]']'CPD-536[c]','GLYCEROL[p]','GLYCEROL[c]',
met_of_interest = ['TAURINE[c]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
rxns = ['RXN0-2162']

rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

,flux,is_new,kinetic
RXN0-2162,-0.0,New Reactions,False


In [791]:
DIMETHYLBENZIMIDAZOLE[CCO-PERI-BAC]S_met

SyntaxError: invalid syntax (1544789365.py, line 1)

In [77]:
homeostatic_cp2

2-3-DIHYDROXYBENZOATE[c]    18.0
2-KETOGLUTARATE[c]          46.0
2-PG[c]                     12.0
2K-4CH3-PENTANOATE[c]       18.0
4-AMINO-BUTYRATE[c]         38.0
                            ... 
NA+[p]                      13.0
OXYGEN-MOLECULE[p]          13.0
FE+3[p]                     13.0
CA+2[p]                     13.0
Pi[p]                       13.0
Name: 24, Length: 172, dtype: float64

In [141]:
met_of_interest = ['HIS[e]', 'CPD-58[e]', 'HEXANOATE[e]', 'CPD-3785[e]', 'CPD-3617[e]', 'CPD-13399[e]', 'XYLITOL[e]', 'CPD0-1937[e]', 'CPD0-1123[e]', 'L-LACTATE[e]', 'GLUCOSAMINE[p]', 'ERYTHRITOL[e]', '4-HYDROXY-BUTYRATE[e]', 'CPD-13413[e]', 'CPD0-2460[e]', 'ACETAMIDE[e]', 'CPD-3611[e]', 'ILE[e]', 'LEU[e]', 'PHE[e]', '5-DEHYDROGLUCONATE[p]', 'CPD-3605[e]', 'CPD-13409[e]', 'OXALATE[e]', '5-AMINOPENTANOATE[e]', 'D-TARTRATE[p]', 'D-ARABINOSE[p]', 'DIHYDROXYACETONE[e]', 'L-ARABITOL[e]', 'VAL[e]', 'GLY[e]', 'CPD-3614[e]', 'CPD-230[e]', 'QUINATE[c]', 'TARTRATE[p]', '4-AMINO-BUTYRATE[e]', '4-hydroxybenzoate[e]', 'ITACONATE[e]', 'MET[e]', 'HOMO-SER[e]', 'ACETYL-GLU[e]', 'CPD-24954[e]', 'CPD-110[e]', '5-OXOPROLINE[c]', 'MANNITOL[e]', 'MALONATE[e]', 'CPD-3627[e]', 'DIACETYL[c]', 'METHYL-BETA-D-GALACTOSIDE[e]', 'BUTYRIC_ACID[e]', 'L-ORNITHINE[e]', 'CPD-335[e]', 'CPD-1099[e]', 'ARG[e]', 'LYS[e]', 'CPD-3609[e]', 'CPD-9569[c]', 'CPD-170[e]', 'CPD-1142[e]', '4-HYDROXY-L-PROLINE[e]', 'CPD-3607[e]', 'Ket0-D-Psicose[e]', 'Alpha-lactose[e]', 'CPD-3623[e]', 'CPD-3801[p]', 'HYDROQUINONE-O-BETA-D-GLUCOPYRANOSIDE[e]', 'PUTRESCINE[e]', 'CPD-355[e]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
all_rxns_to_check = list(rxns)# rxn_flux = sim_flux.loc[rxns]
# rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
# rxn_flux

In [142]:
%store all_rxns_to_check

Stored 'all_rxns_to_check' (list)
